In [25]:
import pandas as pd
import re

In [26]:
# Load the unique data
file_path = '..\\data\\unique\\volkskrant_gpt.csv'
unique_data = pd.read_csv(file_path)

# Display the first few rows to understand the structure
print(unique_data.head())
print(unique_data.columns)  # Print column names to verify structure


   Unnamed: 0                                             prompt  \
0           0  De Nederlandse identiteit in boeken: land van ...   
1           1  De Nederlandse identiteit in boeken: land van ...   
2           2  De Nederlandse identiteit in boeken: land van ...   
3           3  De Nederlandse identiteit in boeken: land van ...   
4           4  De Nederlandse identiteit in boeken: land van ...   

                                                 end  \
0  Oranje, die zich in het Wilhelmus niet als reb...   
1  kunnen - de bewerkte heruitgave van een eerder...   
2  Vier boeken zoeken het antwoord.\nThomas von d...   
3  . Zijn In het land der jaknikkers, een bundeli...   
4  genoemde Volkskrantlezer, 'zeuren en zaniken',...   

                                              result  lcs      lcsp  text_id  \
0  Het artikel bespreekt vier boeken die de Neder...   15  0.004069        0   
1  de Nederlandse identiteit probeert te duiden, ...   30  0.004505        0   
2  De verkiezi

In [27]:
# Function to clean and extract the first five sentences from the article
def extract_first_sentences(text):
    # Split the text into lines
    lines = text.split('\n')
    if len(lines) >= 3:
        # Combine lines after the first few to avoid grabbing the title/authors etc
        combined_text = ' '.join(lines[1:])
        # Split combined text into sentences using regex
        sentences = re.split(r'[.!?]', combined_text)
        # Join the first five sentences
        first_sentences = '. '.join(sentences[:2])
        return first_sentences
    return ''

# Process the first 2000 rows and extract the first few sentences from the article
processed_data = []

for index, row in unique_data.head(2000).iterrows():
    prompt = row['prompt']
    processed_prompt = extract_first_sentences(prompt)
    processed_data.append({'text': processed_prompt})


# Create a new DataFrame with the processed data
processed_df = pd.DataFrame(processed_data)
processed_df['text_count'] = 1

# Display the processed DataFrame to verify
print(processed_df.head())


                                                text  text_count
0  Vier boeken zoeken antwoorden Over de toekomst...           1
1  Vier boeken zoeken antwoorden Over de toekomst...           1
2  Vier boeken zoeken antwoorden Over de toekomst...           1
3  Vier boeken zoeken antwoorden Over de toekomst...           1
4  Vier boeken zoeken antwoorden Over de toekomst...           1


In [28]:
# Function to check if the first 5 words of two texts overlap
def first_five_words_overlap(text1, text2):
    words1 = text1.split()[:5]
    words2 = text2.split()[:5]
    return words1 == words2

# Remove duplicates based on the first 5 overlapping words
unique_processed_data = []
for index, row in processed_df.iterrows():
    is_duplicate = False
    for unique_row in unique_processed_data:
        if first_five_words_overlap(row['text'], unique_row['text']):
            is_duplicate = True
            break
    if not is_duplicate:
        unique_processed_data.append(row)

# Create a new DataFrame with unique processed data
unique_processed_df = pd.DataFrame(unique_processed_data)

# Display the unique processed DataFrame
print(unique_processed_df.head())
print(f"Number of unique processed prompts: {len(unique_processed_df)}")


                                                 text  text_count
0   Vier boeken zoeken antwoorden Over de toekomst...           1
5   Populisten in Zweden beschuldigen sociaal-demo...           1
10  Gratis busvervoer ontlokt op A44 geen daverend...           1
15                                                              1
17  Hayarpi Tamrazyan is al sinds 2010 met haar Ar...           1
Number of unique processed prompts: 393


In [29]:
# Remove duplicates
processed_df = unique_processed_df.drop_duplicates()
processed_df = processed_df.dropna()

# Calculate the token length for each prompt
processed_df['Token Length'] = processed_df['text'].apply(lambda x: len(x.split()))

# Calculate the average token length
average_token_length = processed_df['Token Length'].mean()

# Display the processed DataFrame and average token length
print(processed_df.head())
print(f"Average token length: {average_token_length}")


                                                 text  text_count  \
0   Vier boeken zoeken antwoorden Over de toekomst...           1   
5   Populisten in Zweden beschuldigen sociaal-demo...           1   
10  Gratis busvervoer ontlokt op A44 geen daverend...           1   
15                                                              1   
17  Hayarpi Tamrazyan is al sinds 2010 met haar Ar...           1   

    Token Length  
0             19  
5             38  
10            34  
15             0  
17            43  
Average token length: 35.651399491094146


In [30]:
processed_df['text'][0]

'Vier boeken zoeken antwoorden Over de toekomst van welk land gáán de verkiezingen eigenlijk.  Vier boeken zoeken het antwoord'

In [31]:
df = unique_processed_df[['text', 'text_count']]
df = df[df['text'].str.len() >= 100]
df

,text,text_count
0,Vier boeken zoeken antwoorden Over de toekomst...,1
5,Populisten in Zweden beschuldigen sociaal-demo...,1
10,Gratis busvervoer ontlokt op A44 geen daverend...,1
17,Hayarpi Tamrazyan is al sinds 2010 met haar Ar...,1
20,bellen metlandbouwverslaggever pieter hotse sm...,1
...,...,...
1973,Roeien: Van Eupen soeverein naar halve finale ...,1
1978,"Guido van Woerkom, nu nog baas van de ANWB, is...",1
1983,Generaal VS: kosten JSF moeten zakken De koste...,1
1993,"Redactie 12 januari 2016, 20:30 Duitsers en Pe...",1


In [32]:
df.to_csv('..\\data\\unique\\text_unique_volkskrant.csv')